NFL

Predicting overall fantasy points throughout the season based on pre-season performance

Predicting fantasy points at the end of a game based on performance at any given amount of minutes into the game

(both of these predictions should be done separetely for each position)

First I will try to just make these simple multiple regression problems and see how accurate I can get with that

Creating functions to scrape preseason and in season data from pro football reference

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

# Function to scrape pre-season data
def scrape_preseason_data(year):
    url = f'https://www.pro-football-reference.com/years/{year}/preseason.htm'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table and convert it to a DataFrame
    table = soup.find('table', {'id': 'preseason'})
    df = pd.read_html(StringIO(str(table)))[0]
    return df

# Function to scrape in-game data
def scrape_in_game_data(year):
    url = f'https://www.pro-football-reference.com/years/{year}/games.htm'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table and convert it to a DataFrame
    table = soup.find('table', {'id': 'games'})
    df = pd.read_html(StringIO(str(table)))[0]
    return df


Get pre-season and in-game data for the 2023 season

In [7]:
preseason_df = scrape_preseason_data(2023)
in_game_df = scrape_in_game_data(2023)

# Display the first few rows of the DataFrames
print(preseason_df.head())
print(in_game_df.head())

   Week  Day Unnamed: 2                  VisTm  PF Unnamed: 5  \
0   NaN  Thu   August 3          New York Jets  16          @   
1   1.0  Thu  August 10         Houston Texans  20          @   
2   1.0  Thu  August 10      Minnesota Vikings  13          @   
3   1.0  Fri  August 11      Green Bay Packers  36          @   
4   1.0  Fri  August 11  Washington Commanders  17          @   

                 HomeTm  Pts  
0      Cleveland Browns   21  
1  New England Patriots    9  
2      Seattle Seahawks   24  
3    Cincinnati Bengals   19  
4      Cleveland Browns   15  
  Week  Day        Date    Time             Winner/tie Unnamed: 5  \
0    1  Thu  2023-09-07  8:20PM          Detroit Lions          @   
1    1  Sun  2023-09-10  1:00PM        Atlanta Falcons        NaN   
2    1  Sun  2023-09-10  1:00PM       Cleveland Browns        NaN   
3    1  Sun  2023-09-10  1:00PM   Jacksonville Jaguars          @   
4    1  Sun  2023-09-10  1:00PM  Washington Commanders        NaN   

        

Scrape individual player data

In [8]:
def scrape_player_data(year):
    url = f'https://www.pro-football-reference.com/years/{year}/fantasy.htm'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table and convert it to a DataFrame
    table = soup.find('table', {'id': 'fantasy'})
    df = pd.read_html(StringIO(str(table)))[0]

    # Clean the DataFrame (remove multi-level headers, etc.)
    df.columns = df.columns.droplevel(0)  # Drop the first header level
    df = df.rename(columns={'Unnamed: 0_level_1': 'Player'})  # Rename player column
    df = df[df['Player'] != 'Player']  # Remove extra header rows
    return df


Get player data for the 2023 season

In [9]:
player_df = scrape_player_data(2023)

  Rk                 Player   Tm FantPos Age   G  GS  Cmp  Att   Yds  ...  TD  \
0  1  Christian McCaffrey*+  SFO      RB  27  16  16    0    0     0  ...  21   
1  2          CeeDee Lamb*+  DAL      WR  24  17  17    0    0     0  ...  14   
2  3             Josh Allen  BUF      QB  27  17  17  385  579  4306  ...  15   
3  4          Tyreek Hill*+  MIA      WR  29  16  16    0    0     0  ...  13   
4  5           Jalen Hurts*  PHI      QB  25  17  17  352  538  3858  ...  15   

   2PM  2PP FantPt    PPR   DKPt   FDPt  VBD PosRank OvRank  
0  NaN  NaN    324  391.3  399.3  357.8  157       1      1  
1    1  NaN    268  403.2  411.2  335.7  131       1      2  
2  NaN    3    393  392.6  420.6  410.6  122       1      3  
3  NaN  NaN    257  376.4  380.4  316.9  120       2      4  
4  NaN  NaN    357  356.8  382.8  371.8   89       2      5  

[5 rows x 33 columns]


In [12]:
# Display the first few rows of the DataFrame
print(player_df.head())

# Display the variable names of the DataFrame
print(player_df.columns)

  Rk                 Player   Tm FantPos Age   G  GS  Cmp  Att   Yds  ...  TD  \
0  1  Christian McCaffrey*+  SFO      RB  27  16  16    0    0     0  ...  21   
1  2          CeeDee Lamb*+  DAL      WR  24  17  17    0    0     0  ...  14   
2  3             Josh Allen  BUF      QB  27  17  17  385  579  4306  ...  15   
3  4          Tyreek Hill*+  MIA      WR  29  16  16    0    0     0  ...  13   
4  5           Jalen Hurts*  PHI      QB  25  17  17  352  538  3858  ...  15   

   2PM  2PP FantPt    PPR   DKPt   FDPt  VBD PosRank OvRank  
0  NaN  NaN    324  391.3  399.3  357.8  157       1      1  
1    1  NaN    268  403.2  411.2  335.7  131       1      2  
2  NaN    3    393  392.6  420.6  410.6  122       1      3  
3  NaN  NaN    257  376.4  380.4  316.9  120       2      4  
4  NaN  NaN    357  356.8  382.8  371.8   89       2      5  

[5 rows x 33 columns]
Index(['Rk', 'Player', 'Tm', 'FantPos', 'Age', 'G', 'GS', 'Cmp', 'Att', 'Yds',
       'TD', 'Int', 'Att', 'Yds', 'Y/A'

Preprocess the data to ensure it's suitable for regression analysis

In [ ]:
# Clean the DataFrame (remove unnecessary columns, handle missing values, etc.)
player_df = player_df[['Player', 'Tm', 'FantPos', 'G', 'Cmp', 'Att', 'Yds', 'TD', 'Int', 'Att', 'Yds', 'TD', 'Tgt', 'Rec', 'Yds', 'TD', 'FantPt']]

# Rename columns for clarity
player_df = player_df.rename(columns={'Tm': 'Team', 'FantPos': 'Position', 'G': 'Games', 'Cmp': 'PassingCompletions', 'Att': 'PassingAttempts', 'Yds': 'PassingYards', 'TD': 'PassingTD', 'Int': 'PassingInt', 'Att': 'RushingAttempts', 'Yds': 'RushingYards', 'TD': 'RushingTD', 'Tgt': 'ReceivingTargets', 'Rec': 'Receptions', 'Yds': 'ReceivingYards', 'TD': 'ReceivingTD', 'FantPt': 'FantasyPoints'})

In [ ]:
# Store the current DataFrame as a CSV file
player_df.to_csv('player_data.csv', index=False)

In [20]:
# Display the new variable names of the DataFrame
print(player_df.columns)

# Give each player a unique identifier
player_df['PlayerID'] = player_df['Player'].str.replace('[^a-zA-Z]', '')  # Remove non-alphabetic characters
player_df['PlayerID'] = player_df['PlayerID'].str.lower()  # Convert to lowercase
player_df['PlayerID'] = player_df['PlayerID'].str[:5]  # Take the first 5 characters
player_df['PlayerID'] = player_df['PlayerID'] + player_df.groupby('PlayerID').cumcount().astype(str)  # Add a count to handle duplicates

# Display the new player ID column
print(player_df[['Player', 'PlayerID']])

# Convert columns to appropriate data types
player_df = player_df.apply(pd.to_numeric, errors='ignore')
player_df['FantasyPoints'] = player_df['FantasyPoints'].astype(float)

# Handle missing values (e.g., fill with 0 or use appropriate imputation method)
player_df = player_df.fillna(0)

# Display the cleaned DataFrame
print(player_df.head())

Index(['Player', 'Team', 'Position', 'Games', 'PassingCompletions',
       'RushingAttempts', 'RushingAttempts', 'ReceivingYards',
       'ReceivingYards', 'ReceivingYards', 'ReceivingTD', 'ReceivingTD',
       'ReceivingTD', 'ReceivingTD', 'PassingInt', 'RushingAttempts',
       'RushingAttempts', 'ReceivingYards', 'ReceivingYards', 'ReceivingYards',
       'ReceivingTD', 'ReceivingTD', 'ReceivingTD', 'ReceivingTD',
       'ReceivingTargets', 'Receptions', 'ReceivingYards', 'ReceivingYards',
       'ReceivingYards', 'ReceivingTD', 'ReceivingTD', 'ReceivingTD',
       'ReceivingTD', 'FantasyPoints', 'PlayerID'],
      dtype='object')
                    Player PlayerID
0    Christian McCaffrey*+   chris0
1            CeeDee Lamb*+   ceede0
2               Josh Allen   josh 0
3            Tyreek Hill*+   tyree0
4             Jalen Hurts*   jalen0
..                     ...      ...
647             Kyle Allen   kyle 4
648           Deon Jackson   deon 0
650            David Wells   david

C:\Users\elifu\AppData\Local\Temp\ipykernel_26168\1822676209.py:14: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  player_df = player_df.apply(pd.to_numeric, errors='ignore')
